In [3]:
!pip3 install langchain_community

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_community-0.0.31-py3-none-any.whl (1.9 MB)
  Using cached langsmith-0.1.40-py3-none-any.whl (87 kB)
     |████████████████████████████████| 174 kB 510 kB/s eta 0:00:01
     |████████████████████████████████| 388 kB 495 kB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 391 kB/s eta 0:00:01     |███████████████████████▍        | 10.2 MB 357 kB/s eta 0:00:11
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached langchain_core-0.1.40-py3-none-any.whl (276 kB)
     |████████████████████████████████| 2.1 MB 426 kB/s eta 0:00:01
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
     |████████████████████████████████| 53 kB 389 kB/s eta 0:00:01
     |████████████████████████████████| 81 kB 563 kB/s eta 0:00:01
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached typing_inspect-0.9.0-py

In [4]:
from langchain_community.llms import Ollama

/Users/zurielcevada/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nThe first man on the moon was Neil Armstrong. He stepped foot on the lunar surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first person to walk on the moon.'

In [6]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'\nCertainly! Here are the categories for each of the expenses you provided:\n\n* Spotify AB by Adyen - Entertainment\n* Beta Boulders Ams Amsterdam Nld - Sports\n* ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD - Food and Beverage\n* Bistro Bar Amsterdam - Food and Beverage\n\nI hope this helps! Let me know if you have any other questions.'

### Read transaction data

In [8]:
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 382 kB/s eta 0:00:01
     |████████████████████████████████| 505 kB 500 kB/s eta 0:00:01
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [26]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [28]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

31

In [30]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

In [31]:
response = llm.invoke ("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words." + ", ". join(unique_transactions[1:30]))

In [33]:
response = response.split('\n')

In [34]:
response

['Certainly! Here are the appropriate categories for each expense:',
 '',
 '1. Spotify AB by Adyen - Entertainment',
 '2. Beta Boulders Ams Amsterdam Nld - Sport',
 '3. Tesco Breda - Grocery',
 '4. Monthly Appartment Rent - Housing',
 '5. Vishandel Sier Amsterdam - Food',
 '6. Selling Paintings - Art/Craft',
 '7. Tk Maxx Amsterdam Da - Shopping',
 '8. Consulting - Business',
 '9. Aidsfonds - Charity',
 '10. Tls Bv Inz Ov-Chipkaart - Transportation',
 '11. Etos Amsterdam - Food',
 '12. Beta Boulders Ams Amsterdam - Fitness',
 '13. Salary - Personal',
 '14. Bouldermuur Bv Amsterdam - Business',
 '15. Birtat Restaurant Amsterdam - Food',
 '16. Freelancing - Work',
 '17. Tikkie - Finance',
 '18. Blogging - Personal',
 '19. Taxi Utrecht - Transportation',
 '20. Apple Services - Technology',
 '21. Amazon Lux - Shopping',
 '22. Classpass* Monthly - Fitness',
 '23. Audible Uk AdblCo/Pymt Gbr - Entertainment',
 '',
 'Note: Some of the expenses could fit into multiple categories, but I have grou

### Categorise bank transactions with Llama2

In [12]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [13]:
# Output validation
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."

# Test validation
ResponseChecks(data = ['Hello - World', 'Hello - there!'])

ResponseChecks(data=None)

In [14]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    # Keep only the lines in between blank lines (removing the explaination lines at the beginning and end of the response)
    blank_indexes = [index for index in range(len(response)) if response[index] == '']
    if len(blank_indexes) == 1:
        response = response[(blank_indexes[0] + 1):]
    else:
        response = response[(blank_indexes[0] + 1) : blank_indexes[1]]

    # Print response and validate if it is in the correct format
    print(response)
    ResponseChecks(data = response)
    
    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [15]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Of course! Here are the expenses you provided along with an appropriate category for each one:']


ValidationError: 1 validation error for ResponseChecks
data
  Assertion failed, String does not contain hyphen. [type=assertion_error, input_value=['Of course! Here are the...category for each one:'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.6/v/assertion_error

In [16]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()
max_tries = 7

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # Try and validate output, if it fails, try again for max_tries=7 times
    for j in range(1, max_tries):
        try:
            categories_df = categorize_transactions(transaction_names, llm)
            categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
            
        except:
            if j < max_tries:
                continue
            else:
                raise Exception(f"Cannot categorise transactions indexes {i} to {i+1}.")
        break

['1. Belastingdienst - Taxes', '2. Tesco Breda - Groceries', '3. Monthly Appartment Rent - Housing', '4. Vishandel Sier Amsterdam - Food', '5. Selling Paintings - Art/Craft', '6. Spotify Ab By Adyen - Entertainment', '7. Tk Maxx Amsterdam Da - Shopping', '8. Consulting - Business', '9. Aidsfonds - Charity', '10. TLS BV Inz Ov-Chipkaart - Transportation', '11. Etos Amsterdam - Health', '12. Beta Boulders Ams Amsterdam - Sports', '13. Salary - Personal', '14. Bouldermuur Bv Amsterdam - Business', '15. Birtat Restaurant Amsterdam - Food', '16. Freelancing - Work', '17. Tikkie - Finance', '18. Blogging - Personal', '19. Taxi Utrecht - Transportation', '20. Apple Services - Tech', '21. Amazon Lux - Shopping', '22. Classpass* Monthly - Fitness', '23. Audible Uk AdblCo/Pymt Gbr - Entertainment']


In [17]:
categories_df_all.head()

,Transaction vs category,Transaction,Category
0,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
1,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
2,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
3,4. Vishandel Sier Amsterdam - Food,4. Vishandel Sier Amsterdam,Food
4,5. Selling Paintings - Art/Craft,5. Selling Paintings,Art/Craft


In [50]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [18]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Taxes', 'Groceries', 'Housing', 'Food', 'Art/Craft',
       'Entertainment', 'Shopping', 'Business', 'Charity',
       'Transportation', 'Health', 'Sports', 'Personal', 'Work',
       'Finance', 'Tech', 'Fitness'], dtype=object)

In [19]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [20]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

,Transaction vs category,Transaction,Category
0,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
1,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
2,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
3,4. Vishandel Sier Amsterdam - Food,4. Vishandel Sier Amsterdam,Food and Drinks
4,5. Selling Paintings - Art/Craft,5. Selling Paintings,Art/Craft
5,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
6,7. Tk Maxx Amsterdam Da - Shopping,7. Tk Maxx Amsterdam Da,Shopping
7,8. Consulting - Business,8. Consulting,Business
8,9. Aidsfonds - Charity,9. Aidsfonds,Charity
9,10. TLS BV Inz Ov-Chipkaart - Transportation,10. TLS BV Inz Ov-Chipkaart,Transportation


In [21]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN


In [22]:
import plotly.express as px

ModuleNotFoundError: No module named 'plotly'

In [45]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)